In [27]:
#!pip install openai
#!pip install pandas
#!pip install numpy

#!pip install tqdm
#openai 라이브러리 버젼확인



In [33]:
import pandas as pd
import random
import os
from tqdm import tqdm
from openai import OpenAI

client = OpenAI(
    # This is the default and can be omitted
    api_key=os.environ.get("OPENAI_API_KEY"),
)


In [40]:
# OpenAI API 호출 테스트
response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "What is the capital of France?"}
    ],
    max_tokens=400,
    temperature=0.1
)

#Answer 출력하기
print(response.choices[0].message.content)


The capital of France is Paris.


In [41]:


# CSV 파일 불러오기 (Jupyter Notebook에서 사용할 파일 경로로 변경)
file_path = 'data.csv'  # 파일 경로를 지정하세요
data = pd.read_csv(file_path, encoding='utf-8-sig')  # 한국어 인코딩을 위해 'utf-8' 추가

#data = data.sample(n=5, random_state=42)


In [42]:
data.shape

(10000, 7)

In [44]:
data.head()

,ID,Q15,Major Classification,Minor Classification,Positive topics,Negative topics,Unnamed: 6
0,1,별로 특별하지 않다,광고 전반,평범함,NaN,특별하지 않음,NaN
1,2,밝은 느낌이 드는 광고다,시각적 요소,긍정적 평가,밝은 느낌,NaN,NaN
2,3,평범,Not enough data,Not enough data,NaN,NaN,NaN
3,4,모델이 어울린다,모델,호감도 높음,모델|어울림,NaN,NaN
4,5,모델과 광고 문구,모델,호감도 높음,모델|광고 문구,NaN,NaN


In [ ]:

# GPT-4 API를 사용하여 품질 점수 및 평가 코멘트를 생성하는 함수
def evaluate_quality(row):

    #한국어 Token = 약 450개
    k_prompt = f"""다음은 광고에 대한 사용자 피드백입니다. 이 피드백에 대해 라벨링된 데이터를 평가해주세요.
    피드백: "{row['Q15']}"
    대분류 라벨: "{row['Major Classification']}"
    소분류 라벨: "{row['Minor Classification']}"
    긍정 토픽: "{row['Positive topics']}"
    부정 토픽: "{row['Negative topics']}"
    
    1. 출력 항목은 "품질 점수", "품질 평가 설명" 두 항목만 출력해주세요
    2. "품질 점수" 1점에서 5점 사이의 라벨링 품질 점수를 제공해주세요 (1점은 매우 나쁨, 5점은 매우 좋음).
    3. "품질 점수" 출력은 평가 점수(1~5)만 출력해 주세요
    4. "품질 평가 설명"  품질 점수가 측정된 이유를 설명해주세요
    5. "품질 평가 설명"은 "품질점수"가 1,2,3 점일 때만 이유를 출력해주고 4,5점은 "-" 를 출력하세요
    6. 출력 시 타이틀은 출력하지 말고 품질점수와 품질평가설명만 출력하세요.
    7. "품질 평가 설명"은 국문으로 해주세요
    """
    #영문토큰 Token = 약 240
    prompt = f"""
     Below is the response data from the ad video evaluation survey, 
     labelled with the ad factor Major and Minor categories and positive and negative topics. 
     Please rate the quality of the labelled data.

    response data: "{row['Q15']}"
    Major categories: "{row['Major Classification']}"
    Minor categories: "{row['Minor Classification']}"
    positive topics: "{row['Positive topics']}"
    negative topics: "{row['Negative topics']}"
    
    1. only two output items, "Quality Score" and "Quality Description" 
    2. "Quality Score" Provide a labelled quality score between 1 and 5 (1 being very poor and 5 being very good) 
    3. "Quality Score" output only the assessment score (1-5) 
    4. "Quality Description" Explain why the quality score was measured 
    5. "Quality Descriptions" only displays the reason when "Quality Score" is 1,2,3, and "-" for 4,5 
    6. "Quality Descriptions" should be no more than 20 characters long
    7. Do not print the title when printing, only the quality score and quality assessment explanation 
    8. "Quality Assessment Explanation" should be in Korean
    """
    


    #print("=======================")
    #print(prompt)

    
    # GPT-4 API 호출 (새로운 API 방식 사용)
    response = client.chat.completions.create(
        model="gpt-4o",  # "gpt-4o"에서 "gpt-4"로 수정
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=400,
        temperature=0.1  # 생성되는 답변의 창의성 수준 (필요에 따라 조정 가능)
    )
    
    # GPT 응답에서 품질 점수 및 코멘트를 추출
    gpt_output = response.choices[0].message.content.strip()  # 응답 구조 수정
    
    # 품질 점수와 코멘트 분리 (점수가 첫 번째, 코멘트는 그 뒤)
    lines = gpt_output.split("\n")
    quality_score = lines[0].strip()
    quality_comment = " ".join(lines[1:]).strip() if len(lines) > 1 else "-"

    # 반환 (품질 점수, 품질 평가 설명)
    return quality_score, quality_comment

# 데이터 추출 갯수
n_samples = 200
# 데이터에서 랜덤하게 n_samples개의 샘플을 추출
#sampled_data = data.sample(n=n_samples, random_state=42)
sampled_data = data.sample(n=n_samples) #랜덤을 고정하지 않음


# tqdm을 사용하여 진행율 표시
results = []
for _, row in tqdm(sampled_data.iterrows(), total=sampled_data.shape[0], desc="Evaluating quality"):
    results.append(evaluate_quality(row))

# 결과를 DataFrame에 추가
sampled_data[['QualityScore', 'QualityScoreComment']] = pd.DataFrame(results, index=sampled_data.index)

# 평가 결과를 포함한 데이터를 CSV 파일로 저장 (한글 인코딩 문제 해결)
output_file_path = 'labeled_data_with_quality_scores.csv'
sampled_data.to_csv(output_file_path, index=False, encoding='utf-8-sig')  # 'utf-8-sig'로 인코딩 




In [39]:
output_file_path

'labeled_data_with_quality_scores.csv'